In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
from sentence_transformers import SentenceTransformer, util
import os
from groq import Groq


In [2]:
CYAN = '\033[96m'
YELLOW = '\033[93m'
PINK= '\033[95m'
NEON_GREEN='\033[92m'
RESET_COLOUR='\033[0m'

ollama Api Client

In [3]:
client = Groq(
    api_key="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",
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
from sentence_transformers import SentenceTransformer, util
import os
from groq import Groq

CYAN = '\033[96m'
YELLOW = '\033[93m'
PINK = '\033[95m'
NEON_GREEN = '\033[92m'
RESET_COLOUR = '\033[0m'

client = Groq(
    api_key="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",
)

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

def get_relevant_context(user_input, vault_embeddings, vault_content, model, top_k=3):
    if vault_embeddings.nelement() == 0:
        return []
    input_embedding = model.encode([user_input])
    cos_scores = util.cos_sim(input_embedding, vault_embeddings)[0]
    top_k = min(top_k, len(cos_scores))
    top_indices = torch.topk(cos_scores, k=top_k)[1].tolist()
    relevant_context = [vault_content[idx].strip() for idx in top_indices]
    return relevant_context

def groq_chat(user_input, system_message, vault_embeddings, vault_content, model):
    relevant_context = get_relevant_context(user_input, vault_embeddings, vault_content, model)
    if relevant_context:
        context_str = "\n".join(relevant_context)
        print("Context from RAG: \n\n" + CYAN + context_str + RESET_COLOUR)
    else:
        print(CYAN + "No relevant context found." + RESET_COLOUR)

    user_input_with_context = user_input
    if relevant_context:
        user_input_with_context = context_str + "\n\n" + user_input

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input_with_context}
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192"
    )

    return chat_completion.choices[0].message.content

model = SentenceTransformer("all-MiniLM-L6-v2")
vault_content = []
if os.path.exists("./dataset.txt"):
    with open("./dataset.txt", "r", encoding='utf-8') as vault_file:
        vault_content = vault_file.readlines()

vault_embeddings = model.encode(vault_content) if vault_content else []
vault_embeddings_tensor = torch.tensor(vault_embeddings)
print("Embeddings for each line in the vault:")
print(vault_embeddings_tensor)

user_input = input(YELLOW + "Ask a question about your documents: " + RESET_COLOUR)
system_message = "You are a helpful assistant that is an expert at extracting the most useful information from a given text"
response = groq_chat(user_input, system_message, vault_embeddings_tensor, vault_content, model)
print(NEON_GREEN + "Groq Response: \n\n" + response + RESET_COLOUR)


Embeddings for each line in the vault:
tensor([[-0.1010,  0.0252,  0.0621,  ..., -0.0056, -0.0635, -0.0224],
        [-0.1188,  0.0483, -0.0025,  ...,  0.1264,  0.0465, -0.0157],
        [-0.0105, -0.0493, -0.0672,  ..., -0.0331, -0.1459,  0.0106],
        [-0.1188,  0.0483, -0.0025,  ...,  0.1264,  0.0465, -0.0157],
        [ 0.0334,  0.0579,  0.0395,  ...,  0.0101, -0.0631, -0.0058],
        [-0.1188,  0.0483, -0.0025,  ...,  0.1264,  0.0465, -0.0157]])
